In [1]:
# -*- coding: utf8 -*-
import codecs,glob
import features
import re
import pandas as pd
import numpy as np
import matplotlib as mplt
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import itertools as it
import networkx as nx
import pickle,yaml
import random as rd
#%pylab inline
#pd.options.display.mpl_style = 'default'
debug=False

In [2]:
import unicodedata
def listerUnicode(chaine):
    result=[]
    comb=False
    prec=u""
    for char in chaine:
        print char,
        if unicodedata.combining(char):
            result.append(prec+char)
            prec=""
        else:
            result.append(prec)
            prec=char
    result.append(prec)
    return [r for r in result if r!=""]

In [3]:
ipaIn = listerUnicode(u"SZNêôârEHO")
ipaOut= [u"ʃ",u"ʒ",u"ŋ",u"ɛ̃",u"ɔ̃",u"ɑ̃",u"ʁ",u"ɛ",u"ɥ",u"ɔ"]
toipa = dict(zip(ipaIn, ipaOut))

S Z N ê ô â r E H O


In [4]:
def coderIPA(chaine,table=toipa):
    result=chaine
    for k in table:
        result=result.replace(k,table[k])
    return result

print coderIPA(u"SâZ piNpôN")

ʃɑ̃ʒ piŋpɔ̃ŋ


# Run a function every x seconds

https://stackoverflow.com/questions/474528/what-is-the-best-way-to-repeatedly-execute-a-function-every-x-seconds-in-python

In [370]:
from threading import Timer

class RepeatedTimer(object):
    def __init__(self, interval, function, *args, **kwargs):
        self._timer     = None
        self.interval   = interval
        self.function   = function
        self.args       = args
        self.kwargs     = kwargs
        self.is_running = False
        self.start()

    def _run(self):
        self.is_running = False
        self.start()
        self.function(*self.args, **self.kwargs)

    def start(self):
        if not self.is_running:
            self._timer = Timer(self.interval, self._run)
            self._timer.start()
            self.is_running = True

    def stop(self):
        self._timer.cancel()
        self.is_running = False

# def progressMessage(a,b):
#     print "%d,%d" % (a,b)

# Faire le tableau de paradigmes de la base de verbes

In [5]:
rep="/Volumes/BroadDisk/Copies-iMac-GB/2015-Data/"
dfGold=pd.read_pickle(rep+"MGC-171229-Verbes3.pkl")

In [14]:
dfLex3=dfGold[dfGold["freq"]>10000]
dfLex3.head()

,ortho,phono,ext,cs,ms,vs,lexeme,freq,prob,case
40,a,a,NaN,V,3S,pi,avoir,7.167735e+11,2.076214e-02,pi3S
45,abaissa,abEsa,NaN,V,3S,ai,abaisser,2.640001e+08,7.647056e-06,ai3S
46,abaissai,abEsE,NaN,V,1S,ai,abaisser,1.700001e+07,4.924243e-07,ai1S
47,abaissaient,abEsE,"t""",V,3P,ii,abaisser,4.100005e+07,1.187612e-06,ii3P
50,abaissait,abEsE,"t""",V,3S,ii,abaisser,2.520003e+08,7.299467e-06,ii3S


In [8]:
dfPar3=pd.pivot_table(dfLex3, values="phono", index=['lexeme'], columns=['case'], aggfunc=lambda x: ",".join(x)).reset_index().reindex()
cases=dfPar3.columns[2:].tolist()
dfPar3.head()

case,lexeme,ai1P,ai1S,ai2P,ai2S,ai3P,ai3S,fi1P,fi1S,fi2P,...,ppFP,ppFS,ppMP,ppMS,ps1P,ps1S,ps2P,ps2S,ps3P,ps3S
0,abaisser,NaN,abEsE,NaN,NaN,abEsEr,abEsa,NaN,abEs6rE,NaN,...,abEse,abEse,abEse,abEse,NaN,NaN,NaN,NaN,NaN,NaN
1,abandonner,abâdOnam,abâdOnE,NaN,abâdOna,abâdOnEr,abâdOna,abâdOn6rô,abâdOn6rE,abâdOn6re,...,abâdOne,abâdOne,abâdOne,abâdOne,NaN,NaN,NaN,abâdOn,abâdOn,abâdOn
2,abasourdir,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,abazurdi,abazurdi,abazurdi,NaN,NaN,NaN,NaN,NaN,NaN
3,abattre,NaN,abati,NaN,NaN,abatir,abati,abatrô,abatrE,abatre,...,abaty,abaty,abaty,abaty,NaN,abat,NaN,abat,NaN,abat
4,abdiquer,NaN,abdikE,NaN,NaN,NaN,abdika,NaN,NaN,NaN,...,NaN,abdike,NaN,abdike,NaN,NaN,NaN,NaN,NaN,NaN


# Calculer la constante et la différence d'une paire de formes

In [9]:
def diff(mot1,mot2):
    result=[]
    diff1=""
    diff2=""
    same=""
    vide="."
    lmax=max(len(mot1),len(mot2))
    lmin=min(len(mot1),len(mot2))
    for index in range(lmax):
        if index < lmin:
            if mot1[index]!=mot2[index]:
                diff1+=mot1[index]
                diff2+=mot2[index]
                same+=vide
            else:
                same+=mot1[index]
                diff1+=vide
                diff2+=vide
        elif index < len(mot1):
            diff1+=mot1[index]
        elif index < len(mot2):
            diff2+=mot2[index]
    diff1=diff1.lstrip(".")
    diff2=diff2.lstrip(".")
    return (same,diff1,diff2,diff1+"_"+diff2)
#    return (diff1+"-"+diff2)

In [10]:
diff("abad","abod")

('ab.d', 'a.', 'o.', 'a._o.')

# Extraire le radical et le patron d'un paradigme verbal

In [11]:
def removeRad(forme,radical):
    result=""
    frontiere=False
    for nEl,element in enumerate(forme):
        if not frontiere and (nEl>=len(radical) or element!=radical[nEl]):
            frontiere=True
            result+=element
        elif frontiere and nEl<len(radical) and radical[nEl]!=".":
            result+="."
        elif frontiere:
            result+=element
    return result

def extractRad(formes):
    radical=formes[0]
    for forme in formes[1:]:
        radical=diff(radical,forme)[0]
    return radical

def makePatAlt(lPar,radical):
    patAlt={}
    for case in lPar:
        patAlt[case]=removeRad(lPar[case],radical)
    return patAlt

In [12]:
formes=u"m@ner mEn@r m@nôr mEn@rô".split(" ")
extractRad(formes)
removeRad(u"abEsâ",u'abEs')

u'\xe2'

# Calcul des patrons bruts

In [13]:
def patronLigne(row):
    patron={}
    formes=[]
    lPar={}
    patAlt={}
    for case in cases:
        if isinstance(row[case],unicode):
            formes.append(row[case])
            lPar[case]=row[case]
    if len(lPar)>1:
        radical=extractRad(formes)
        patAlt=makePatAlt(lPar,radical)
    return patAlt

In [15]:
dfPar3["patron"]=dfPar3.apply(lambda x: patronLigne(x),axis=1)
dfPar3.head()

case,lexeme,ai1P,ai1S,ai2P,ai2S,ai3P,ai3S,fi1P,fi1S,fi2P,...,ppFS,ppMP,ppMS,ps1P,ps1S,ps2P,ps2S,ps3P,ps3S,patron
0,abaisser,NaN,abEsE,NaN,NaN,abEsEr,abEsa,NaN,abEs6rE,NaN,...,abEse,abEse,abEse,NaN,NaN,NaN,NaN,NaN,NaN,"{u'pP': u'â', u'ppMS': u'e', u'ai3P': u'Er', u..."
1,abandonner,abâdOnam,abâdOnE,NaN,abâdOna,abâdOnEr,abâdOna,abâdOn6rô,abâdOn6rE,abâdOn6re,...,abâdOne,abâdOne,abâdOne,NaN,NaN,NaN,abâdOn,abâdOn,abâdOn,"{u'ii1P': u'jô', u'ii1S': u'E', u'pP': u'â', u..."
2,abasourdir,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,abazurdi,abazurdi,abazurdi,NaN,NaN,NaN,NaN,NaN,NaN,"{u'ppMS': u'', u'inf': u'r', u'ppMP': u'', u'p..."
3,abattre,NaN,abati,NaN,NaN,abatir,abati,abatrô,abatrE,abatre,...,abaty,abaty,abaty,NaN,abat,NaN,abat,NaN,abat,"{u'ii1P': u'tjô', u'ii1S': u'tE', u'pP': u'tâ'..."
4,abdiquer,NaN,abdikE,NaN,NaN,NaN,abdika,NaN,NaN,NaN,...,abdike,NaN,abdike,NaN,NaN,NaN,NaN,NaN,NaN,"{u'ai3S': u'a', u'ii1S': u'E', u'ii3P': u'E', ..."


# Cacul du nombre de lexèmes par patron brut

In [28]:
dfPatronsTemp=dfPar3[["lexeme","patron"]]

In [29]:
dfPatrons=pd.concat([dfPatronsTemp.drop(['patron'], axis=1), dfPatronsTemp['patron'].apply(pd.Series)], axis=1)

In [31]:
dfPatrons["nbCases"]=dfPatrons.notnull().sum(axis=1)
dfPatrons.fillna("-",inplace=True)

In [294]:
dfPatrons.groupby(cases).agg({"lexeme":"count","nbCases":"first"}).sort_values('lexeme', ascending=False).head()

lexeme  \
ai1S ai2P ai2S ai3P ai3S fi1P fi1S fi2P fi2S fi3P fi3S ii1P ii1S ii2P ii2S ii3P ii3S inf is1P is1S is2P is2S is3P is3S pI1P pI2P pI2S pP pc1P pc1S pc2P pc2S pc3P pc3S pi1P pi1S pi2P pi2S pi3P pi3S ppFP ppFS ppMP ppMS ps1P ps1S ps2P ps2S ps3P ps3S           
-    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -   -    -    -    -    -    -    -    -    -    -  -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -        458   
                                                                                         -    -    -    -    -    -    -    -    -    -  -    -    -    -    -    -    -    -    -    -    -    -    -    -    -         -    -    -    -    -    -         69   
                                                                                     -   -    -    -    -    -    -    -    -    -    -  -    -    -    -    -    -    -    -    -    -    -    -    -         -         -    -    -    -    -    -         31   
                                                                                         -    -    -    -    -    -    -    -    -    -  -    -    -    -    -    -    -    -    -    -    -    -                        -    -    -    -    -    -         20   
                                                                                                                                                                                                     -                   -    -    -    -    -    -         20   

                                                                                                                                                                                                                                                        nbCases  
ai1S ai2P ai2S ai3P ai3S fi1P fi1S fi2P fi2S fi3P fi3S ii1P ii1S ii2P ii2S ii3P ii3S inf is1P is1S is2P is2S is3P is3S pI1P pI2P pI2S pP pc1P pc1S pc2P pc2S pc3P pc3S pi1P pi1S pi2P pi2S pi3P pi3S ppFP ppFS ppMP ppMS ps1P ps1S ps2P ps2S ps3P ps3S           
-    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -   -    -    -    -    -    -    -    -    -    -  -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -    -           1  
                                                                                         -    -    -    -    -    -    -    -    -    -  -    -    -    -    -    -    -    -    -    -    -    -    -    -    -         -    -    -    -    -    -           3  
                                                                                     -   -    -    -    -    -    -    -    -    -    -  -    -    -    -    -    -    -    -    -    -    -    -    -         -         -    -    -    -    -    -           3  
                                                                                         -    -    -    -    -    -    -    -    -    -  -    -    -    -    -    -    -    -    -    -    -    -                        -    -    -    -    -    -           6  
                                                                                                                                                                                                     -                   -    -    -    -    -    -           5

In [295]:
dfPatronsBruts=dfPatrons.groupby(cases).agg({"lexeme":"count","nbCases":"first"}).sort_values('nbCases', ascending=False).reset_index()

In [334]:
def compatibiliteComp(c1B,c2B,extra1="-",extra2="-"):
    '''
    c1B et c2B sont les éléments de patron locaux, 
    extra1 et extra2 sont les éléments ajoutés à gauche des patrons
    on ajoute les extras pendant la comparaison et on resépare les éléments à la fin
    '''
#    print c1B,c2B,extra1,extra2
    lMax=max(len(c1B),len(c2B))
    if extra1=="-" or c1B=="-":
        c1=c1B
    else:
        c1=extra1+c1B
    if extra2=="-" or c2B=="-":
        c2=c2B
    else:
        c2=extra2+c2B
    lPossible=True
    if c1B!="-" and c2B!="-":
        longueurAB=len(c1)
        extra=""
        if longueurAB<len(c2):
            extra=c2[:len(c2)-longueurAB]
        elif longueurAB>len(c2):
            longueurAB=len(c2)
            extra=c1[:len(c1)-longueurAB]
        result=extra
        c1Comp=c1[-longueurAB:]
        c2Comp=c2[-longueurAB:]
#        print c1Comp,c2Comp,longueurAB
        lCompatibilite=True
        for c in range(longueurAB):
            if c1Comp[c]==".":
                result+=c2Comp[c]
            elif c2Comp[c]==".":
                result+=c1Comp[c]
            elif c1Comp[c]==c2Comp[c]:
                result+=c1Comp[c]
            else:
                lPossible=False
                break
        comp1=result[-len(c1B):]
        comp2=result[-len(c2B):]
        extra1=result[:-len(c1B)]
        extra2=result[:-len(c2B)]
    elif c1=="-":
        comp1=c2
        comp2=c2[-len(c2B):]
        extra1="-"
        extra2=c2[:-len(c2B)]
        if extra2=="":
            extra2="-"
    else:
        comp1=c1
        comp2=c1[-len(c1B):]
        extra1=c1[:-len(c1B)]
        if extra1=="":
            extra1="-"
        extra2="-"
    
    return lPossible,extra1,comp1,extra2,comp2
    

In [336]:
compatibiliteComp("6.Er","6.Er","-","-")

(True, '', '6.Er', '', '6.Er')

In [298]:
def compatibiliteAB(lDF,a,b):
    rowA=lDF.iloc[a]
    rowB=lDF.iloc[b]
    extra1="-"
    extra2="-"
    lPossible=True
    result=""
    patronAB=[]
    for case in cases:
        lPossible,extra1,comp1,extra2,comp2=compatibiliteComp(rowA[case],rowB[case],extra1,extra2)
        if not lPossible:
            break
        patronAB.append((comp1,comp2))
    result=";".join([extra1+comp1 for (comp1,comp2) in patronAB])
    if not lPossible:
        result=None
    return result

In [312]:
def compatibiliteAB(a,b):
    rowA=a.split(";")
    rowB=b.split(";")
    extra1="-"
    extra2="-"
    lPossible=True
    result=""
    patronAB=[]
    for case in range(len(rowA)):
        lPossible,extra1,comp1,extra2,comp2=compatibiliteComp(rowA[case],rowB[case],extra1,extra2)
        if not lPossible:
            break
        patronAB.append((comp1,comp2))
    result=";".join([extra1+comp1 if comp1!="-" else"-" for (comp1,comp2) in patronAB])
    if not lPossible:
        result=None
    return result

Pb avec les tirets pour les extras

In [360]:
patronA=";".join(dfPatronsBruts[cases].iloc[51].tolist())
patronB=";".join(dfPatronsBruts[cases].iloc[53].tolist())
patronA="6.E;-;6.a;6.Er"
patronB="6.E;-;-;6.Er"
print patronA
print
print patronB
print
print compatibiliteAB(patronA,patronB)

6.E;-;6.a;6.Er

6.E;-;-;6.Er

6.E;-;6.a;6.Er


# Faire un graphe dirigé pour les patrons
1. introduire tous les patrons de base
1. ajouter un lien vers le patronAB si le patron est différent pour A, pour B
1. recommencer avec les sommets

In [369]:
patronA=";".join(dfPatronsBruts[cases].iloc[2510].tolist())
patronB=";".join(dfPatronsBruts[cases].iloc[2916].tolist())
print patronA
print patronB
setA=set([n for n,v in enumerate(patronA.split(";")) if v!="-"])
setB=set([n for n,v in enumerate(patronB.split(";")) if v!="-"])
setAB=setA&setB
setAmoinsB=setA-setB
setBmoinsA=setB-setA
print setA,setB
print setAmoinsB,setBmoinsA
if set():
    print 1
else:
    print 0

-;-;-;-;a;-;-;-;-;-;-;-;-;-;-;E;E;e;-;-;-;-;-;-;-;-;-;â;-;-;-;-;-;-;-;-;-;-;;;e;e;e;e;-;-;-;;-;-
-;-;-;Er;a;-;-;-;-;-;6ra;-;-;-;-;-;-;e;-;-;-;-;-;-;-;-;-;â;-;-;-;-;-;-;-;-;e;-;-;;-;e;e;e;-;-;-;-;-;-
set([4, 38, 39, 40, 41, 42, 43, 47, 15, 16, 17, 27]) set([3, 4, 39, 41, 10, 43, 17, 36, 27, 42])
set([40, 16, 15, 38, 47]) set([10, 3, 36])
0


In [394]:
from datetime import datetime

def progressMessage():
    print "%d,%d\t%s" % (a,b,datetime.now().strftime('%H:%M'))


In [398]:
graphPatrons=nx.DiGraph()
def getSommets(lDiGraph):
    return [k for k,v in lDiGraph.out_degree().iteritems() if v==0]

In [399]:
possibleUnions={}
impossibleUnions=[]
rt = RepeatedTimer(600, progressMessage) # it auto-starts, no need of rt.start()
try:
    for a,b in it.combinations(range(len(dfPatronsBruts)),2):
        patronA=";".join(dfPatronsBruts[cases].iloc[a].tolist())
        patronB=";".join(dfPatronsBruts[cases].iloc[b].tolist())
        if patronA not in graphPatrons:
            graphPatrons.add_node(patronA)
        if patronB not in graphPatrons:
            graphPatrons.add_node(patronB)
        setA=set([n for n,v in enumerate(patronA.split(";")) if v!="-"])
        setB=set([n for n,v in enumerate(patronB.split(";")) if v!="-"])
        if setA&setB:
            patronAB=compatibiliteAB(patronA,patronB)
            if patronAB!=None:
                if patronAB not in possibleUnions:
                    possibleUnions[patronAB]=[]
                possibleUnions[patronAB].append((a,b))
                if patronA!=patronAB:
                    graphPatrons.add_edge(patronA,patronAB)
                if patronB!=patronAB:
                    graphPatrons.add_edge(patronB,patronAB)
        else:
            print "%d,%d : pas de pivot"%(a,b)
            impossibleUnions.append((a,b))
finally:
    rt.stop() # better in a try/finally block to make sure the program ends!

0,4236 : pas de pivot
1,4236 : pas de pivot
2,4236 : pas de pivot
3,4039 : pas de pivot
3,4149 : pas de pivot
3,4201 : pas de pivot
3,4216 : pas de pivot
3,4236 : pas de pivot
4,4236 : pas de pivot
5,4236 : pas de pivot
6,4236 : pas de pivot
7,4236 : pas de pivot
8,4236 : pas de pivot
9,4236 : pas de pivot
10,4236 : pas de pivot
11,4236 : pas de pivot
12,4236 : pas de pivot
13,4236 : pas de pivot
14,4236 : pas de pivot
15,4236 : pas de pivot
16,4236 : pas de pivot
17,4236 : pas de pivot
18,4236 : pas de pivot
19,4236 : pas de pivot
20,4236 : pas de pivot
21,4236 : pas de pivot
22,4236 : pas de pivot
23,4236 : pas de pivot
24,4236 : pas de pivot
25,4236 : pas de pivot
26,4236 : pas de pivot
27,4236 : pas de pivot
28,4236 : pas de pivot
29,4236 : pas de pivot
30,4236 : pas de pivot
31,4236 : pas de pivot
32,4236 : pas de pivot
33,4236 : pas de pivot
34,4236 : pas de pivot
35,4236 : pas de pivot
36,4236 : pas de pivot
37,4236 : pas de pivot
38,4236 : pas de pivot
39,4236 : pas de pivot
40

291,4236 : pas de pivot
292,4236 : pas de pivot
293,4236 : pas de pivot
294,4236 : pas de pivot
295,4141 : pas de pivot
295,4236 : pas de pivot
296,4236 : pas de pivot
297,4236 : pas de pivot
298,4236 : pas de pivot
299,4236 : pas de pivot
300,4236 : pas de pivot
301,4236 : pas de pivot
302,4039 : pas de pivot
302,4149 : pas de pivot
302,4201 : pas de pivot
302,4216 : pas de pivot
302,4236 : pas de pivot
303,4039 : pas de pivot
303,4149 : pas de pivot
303,4201 : pas de pivot
303,4216 : pas de pivot
303,4236 : pas de pivot
304,4236 : pas de pivot
305,4141 : pas de pivot
305,4236 : pas de pivot
306,4236 : pas de pivot
307,4236 : pas de pivot
308,4236 : pas de pivot
309,4236 : pas de pivot
310,4141 : pas de pivot
310,4236 : pas de pivot
311,4236 : pas de pivot
312,4236 : pas de pivot
313,4141 : pas de pivot
313,4236 : pas de pivot
314,4236 : pas de pivot
315,4236 : pas de pivot
316,4236 : pas de pivot
317,4236 : pas de pivot
318,4236 : pas de pivot
319,4236 : pas de pivot
320,4236 : pas d

508,4236 : pas de pivot
509,4236 : pas de pivot
510,4141 : pas de pivot
510,4236 : pas de pivot
511,4236 : pas de pivot
512,4141 : pas de pivot
512,4236 : pas de pivot
513,4236 : pas de pivot
514,4236 : pas de pivot
515,4236 : pas de pivot
516,4236 : pas de pivot
517,4039 : pas de pivot
517,4149 : pas de pivot
517,4201 : pas de pivot
517,4216 : pas de pivot
517,4236 : pas de pivot
518,4039 : pas de pivot
518,4149 : pas de pivot
518,4201 : pas de pivot
518,4216 : pas de pivot
518,4236 : pas de pivot
519,4236 : pas de pivot
520,3901	07:15
520,4141 : pas de pivot
520,4236 : pas de pivot
521,4236 : pas de pivot
522,4236 : pas de pivot
523,4236 : pas de pivot
524,4236 : pas de pivot
525,4236 : pas de pivot
526,4236 : pas de pivot
527,4236 : pas de pivot
528,4141 : pas de pivot
528,4236 : pas de pivot
529,4141 : pas de pivot
529,4236 : pas de pivot
530,4236 : pas de pivot
531,4236 : pas de pivot
532,4141 : pas de pivot
532,4236 : pas de pivot
533,4141 : pas de pivot
533,4236 : pas de pivot
5

697,4236 : pas de pivot
698,4141 : pas de pivot
698,4236 : pas de pivot
699,4141 : pas de pivot
699,4236 : pas de pivot
700,4141 : pas de pivot
700,4236 : pas de pivot
701,4236 : pas de pivot
702,4141 : pas de pivot
702,4236 : pas de pivot
703,4141 : pas de pivot
703,4236 : pas de pivot
704,4141 : pas de pivot
704,4236 : pas de pivot
705,4236 : pas de pivot
706,4141 : pas de pivot
706,4236 : pas de pivot
707,4141 : pas de pivot
707,4236 : pas de pivot
708,4141 : pas de pivot
708,4236 : pas de pivot
709,4039 : pas de pivot
709,4149 : pas de pivot
709,4201 : pas de pivot
709,4216 : pas de pivot
709,4236 : pas de pivot
710,4141 : pas de pivot
710,4236 : pas de pivot
711,4141 : pas de pivot
711,4236 : pas de pivot
712,4141 : pas de pivot
712,4236 : pas de pivot
713,4141 : pas de pivot
713,4236 : pas de pivot
714,4141 : pas de pivot
714,4236 : pas de pivot
715,4039 : pas de pivot
715,4141 : pas de pivot
715,4149 : pas de pivot
715,4201 : pas de pivot
715,4216 : pas de pivot
715,4236 : pas d

868,4149 : pas de pivot
868,4201 : pas de pivot
868,4216 : pas de pivot
868,4236 : pas de pivot
869,4039 : pas de pivot
869,4149 : pas de pivot
869,4201 : pas de pivot
869,4216 : pas de pivot
869,4236 : pas de pivot
870,4141 : pas de pivot
870,4236 : pas de pivot
871,4141 : pas de pivot
871,4236 : pas de pivot
872,4141 : pas de pivot
872,4236 : pas de pivot
873,4141 : pas de pivot
873,4236 : pas de pivot
874,4141 : pas de pivot
874,4236 : pas de pivot
875,4039 : pas de pivot
875,4149 : pas de pivot
875,4201 : pas de pivot
875,4216 : pas de pivot
875,4236 : pas de pivot
876,4141 : pas de pivot
876,4194 : pas de pivot
876,4209 : pas de pivot
876,4210 : pas de pivot
876,4236 : pas de pivot
877,4039 : pas de pivot
877,4149 : pas de pivot
877,4201 : pas de pivot
877,4216 : pas de pivot
877,4236 : pas de pivot
878,4141 : pas de pivot
878,4236 : pas de pivot
879,4236 : pas de pivot
880,4236 : pas de pivot
881,4141 : pas de pivot
881,4236 : pas de pivot
882,4141 : pas de pivot
882,4236 : pas d

...

4171,4201 : pas de pivot
4171,4202 : pas de pivot
4171,4203 : pas de pivot
4171,4204 : pas de pivot
4171,4206 : pas de pivot
4171,4209 : pas de pivot
4171,4210 : pas de pivot
4171,4211 : pas de pivot
4171,4214 : pas de pivot
4171,4215 : pas de pivot
4171,4216 : pas de pivot
4171,4218 : pas de pivot
4171,4219 : pas de pivot
4171,4223 : pas de pivot
4171,4224 : pas de pivot
4171,4225 : pas de pivot
4171,4226 : pas de pivot
4171,4227 : pas de pivot
4171,4228 : pas de pivot
4171,4229 : pas de pivot
4171,4230 : pas de pivot
4171,4231 : pas de pivot
4171,4233 : pas de pivot
4171,4236 : pas de pivot
4172,4173 : pas de pivot
4172,4177 : pas de pivot
4172,4178 : pas de pivot
4172,4179 : pas de pivot
4172,4181 : pas de pivot
4172,4182 : pas de pivot
4172,4183 : pas de pivot
4172,4184 : pas de pivot
4172,4185 : pas de pivot
4172,4186 : pas de pivot
4172,4187 : pas de pivot
4172,4188 : pas de pivot
4172,4189 : pas de pivot
4172,4190 : pas de pivot
4172,4191 : pas de pivot
4172,4192 : pas de pivot


4179,4236 : pas de pivot
4180,4181 : pas de pivot
4180,4182 : pas de pivot
4180,4183 : pas de pivot
4180,4184 : pas de pivot
4180,4185 : pas de pivot
4180,4186 : pas de pivot
4180,4187 : pas de pivot
4180,4188 : pas de pivot
4180,4189 : pas de pivot
4180,4190 : pas de pivot
4180,4192 : pas de pivot
4180,4193 : pas de pivot
4180,4197 : pas de pivot
4180,4198 : pas de pivot
4180,4199 : pas de pivot
4180,4200 : pas de pivot
4180,4201 : pas de pivot
4180,4203 : pas de pivot
4180,4205 : pas de pivot
4180,4207 : pas de pivot
4180,4208 : pas de pivot
4180,4211 : pas de pivot
4180,4214 : pas de pivot
4180,4216 : pas de pivot
4180,4222 : pas de pivot
4180,4231 : pas de pivot
4180,4234 : pas de pivot
4180,4235 : pas de pivot
4180,4236 : pas de pivot
4181,4184 : pas de pivot
4181,4186 : pas de pivot
4181,4188 : pas de pivot
4181,4190 : pas de pivot
4181,4191 : pas de pivot
4181,4192 : pas de pivot
4181,4193 : pas de pivot
4181,4194 : pas de pivot
4181,4195 : pas de pivot
4181,4196 : pas de pivot


4190,4198 : pas de pivot
4190,4200 : pas de pivot
4190,4202 : pas de pivot
4190,4203 : pas de pivot
4190,4206 : pas de pivot
4190,4209 : pas de pivot
4190,4210 : pas de pivot
4190,4211 : pas de pivot
4190,4213 : pas de pivot
4190,4214 : pas de pivot
4190,4215 : pas de pivot
4190,4218 : pas de pivot
4190,4219 : pas de pivot
4190,4223 : pas de pivot
4190,4224 : pas de pivot
4190,4225 : pas de pivot
4190,4226 : pas de pivot
4190,4227 : pas de pivot
4190,4229 : pas de pivot
4190,4230 : pas de pivot
4190,4231 : pas de pivot
4190,4233 : pas de pivot
4190,4236 : pas de pivot
4191,4192 : pas de pivot
4191,4193 : pas de pivot
4191,4197 : pas de pivot
4191,4198 : pas de pivot
4191,4199 : pas de pivot
4191,4200 : pas de pivot
4191,4201 : pas de pivot
4191,4202 : pas de pivot
4191,4203 : pas de pivot
4191,4205 : pas de pivot
4191,4208 : pas de pivot
4191,4211 : pas de pivot
4191,4212 : pas de pivot
4191,4214 : pas de pivot
4191,4216 : pas de pivot
4191,4218 : pas de pivot
4191,4219 : pas de pivot


4203,4212 : pas de pivot
4203,4213 : pas de pivot
4203,4214 : pas de pivot
4203,4215 : pas de pivot
4203,4217 : pas de pivot
4203,4218 : pas de pivot
4203,4219 : pas de pivot
4203,4220 : pas de pivot
4203,4221 : pas de pivot
4203,4222 : pas de pivot
4203,4223 : pas de pivot
4203,4224 : pas de pivot
4203,4226 : pas de pivot
4203,4227 : pas de pivot
4203,4228 : pas de pivot
4203,4230 : pas de pivot
4203,4232 : pas de pivot
4203,4233 : pas de pivot
4203,4234 : pas de pivot
4203,4236 : pas de pivot
4204,4205 : pas de pivot
4204,4207 : pas de pivot
4204,4208 : pas de pivot
4204,4211 : pas de pivot
4204,4212 : pas de pivot
4204,4214 : pas de pivot
4204,4218 : pas de pivot
4204,4219 : pas de pivot
4204,4220 : pas de pivot
4204,4221 : pas de pivot
4204,4222 : pas de pivot
4204,4223 : pas de pivot
4204,4224 : pas de pivot
4204,4225 : pas de pivot
4204,4226 : pas de pivot
4204,4227 : pas de pivot
4204,4229 : pas de pivot
4204,4230 : pas de pivot
4204,4231 : pas de pivot
4204,4232 : pas de pivot


4229,4234 : pas de pivot
4229,4235 : pas de pivot
4229,4236 : pas de pivot
4230,4231 : pas de pivot
4230,4234 : pas de pivot
4230,4235 : pas de pivot
4230,4236 : pas de pivot
4231,4232 : pas de pivot
4231,4233 : pas de pivot
4231,4234 : pas de pivot
4231,4235 : pas de pivot
4231,4236 : pas de pivot
4232,4236 : pas de pivot
4233,4234 : pas de pivot
4233,4235 : pas de pivot
4233,4236 : pas de pivot
4234,4236 : pas de pivot
4235,4236 : pas de pivot


In [405]:
yaml.safe_dump(possibleUnions, file("LEX3-PossibleUnions.yaml", 'w'), encoding='utf-8', allow_unicode=True)

In [ ]:
sommets=getSommets(graphPatrons)

In [402]:
len(sommets)
# for numS,sommet in enumerate(sommets):
#     print numS,sommet
#     print

1044377